### Import librairies

In [3]:
import os
import pandas as pd
import anndata as ad
from FlowCytometryTools import FCMeasurement
import re
import numpy as np
#from list_paths import create_list_of_paths,create_list_of_paths_spec_patients
from plot_pred_vs_true import plot_result

### Create functions to load and concatenate data

In [34]:
def arcsinh_transform(X,cofactor=5):
    return np.arcsinh(X/cofactor)

In [35]:
def load_data_fcs(path,condition_name,cell_type=None):
    data = FCMeasurement(ID="Sample", datafile=path)
    anndata = ad.AnnData(data.data)

    anndata.obs['drug']= condition_name
    if cell_type:
        anndata.obs['cell_type']= cell_type
    anndata.X=arcsinh_transform(anndata.X)
    return anndata

In [6]:
def concatenate_2conditions_data(path_stim,stim_name,path_unstim,unstim_name,outdir_path):
    unstim_anndata = load_data_fcs(path_unstim,unstim_name)
    stim_anndata = load_data_fcs(path_stim,stim_name)

    combined_anndata = stim_anndata.concatenate(unstim_anndata, batch_key="condition", batch_categories=["stim", "control"])
    combined_anndata.write(outdir_path)
    return

In [36]:
def concatenate_2conditions_multiple_data(path_stim_list,stim_name,path_unstim_list,unstim_name,outdir_path):
    comb_unstim_anndata = load_data_fcs(path_unstim_list[0][0],unstim_name,path_stim_list[0][1])
    comb_stim_anndata = load_data_fcs(path_stim_list[0][0],stim_name,path_stim_list[0][1])
    for path_stim in path_stim_list[1:]:
        if path_stim[1]:
            stim_anndata = load_data_fcs(path_stim[0],stim_name,path_stim[1])
        else:
            stim_anndata = load_data_fcs(path_stim[0],stim_name)
        comb_stim_anndata = comb_stim_anndata.concatenate(stim_anndata)
        
    for path_unstim in path_unstim_list[1:]:
        if path_unstim[1]:
            unstim_anndata = load_data_fcs(path_unstim[0],unstim_name,path_unstim[1])
        else:
            unstim_anndata = load_data_fcs(path_unstim[0],unstim_name)
        comb_unstim_anndata = comb_unstim_anndata.concatenate(unstim_anndata)
    combined_anndata=comb_stim_anndata.concatenate(comb_unstim_anndata, batch_key="condition", batch_categories=["stim", "control"])
    
    combined_anndata.write(outdir_path)
    return 

In [8]:
def concatenate_1condition_multiple_data(path_list,condition_name,outdir_path):
    comb_anndata = load_data_fcs(path_list[0],condition_name)

    for path in path_list[1:]:
        new_anndata = load_data_fcs(path,condition_name)
        comb_anndata = comb_anndata.concatenate(new_anndata)
    comb_anndata.write(outdir_path)
    return 

###  Examples

In [25]:
from list_paths import create_list_of_paths2,create_list_of_paths_spec_patients

In [ ]:
perio_stim_list=['TNFa','P. gingivalis','IFNa']
perio_cell_list=['Granulocytes (CD45-CD66+)','B-Cells (CD19+CD3-)','Classical Monocytes (CD14+CD16-)','MDSCs (lin-CD11b-CD14+HLADRlo)','mDCs (CD11c+HLADR+)','pDCs(CD123+HLADR+)','Intermediate Monocytes (CD14+CD16+)','Non-classical Monocytes (CD14-CD16+)','CD56+CD16- NK Cells','CD56loCD16+NK Cells','NK Cells (CD7+)','CD4 T-Cells','Tregs (CD25+FoxP3+)','CD8 T-Cells','CD8-CD4- T-Cells']

In [ ]:
path_unstim_list=create_list_of_paths2(directory='PerioStudy_Complete Plates 1-13_011717 (Clone) - Gated Populations',stimulation='Unstim',cell_type="Granulocytes (CD45-CD66+)",patient_excluded=['P01'])
path_stim_list=create_list_of_paths2(directory='PerioStudy_Complete Plates 1-13_011717 (Clone) - Gated Populations',stimulation="TNFa",cell_type="Granulocytes (CD45-CD66+)",patient_excluded=['P01'])
res=concatenate_2conditions_multiple_data(path_stim_list,"TNFa",path_unstim_list,'Unstim',"perio_data_sherlock/perio_data_sherlock"+"TNFa"+"_"+"Granulocytes (CD45-CD66+)"+"_train.h5ad")

In [27]:
c=0
tot=len(perio_stim_list)*len(perio_cell_list)
for cell_type in perio_cell_list:
    for stim in perio_stim_list:
        path_unstim_list=create_list_of_paths2(directory='PerioStudy_Complete Plates 1-13_011717 (Clone) - Gated Populations',stimulation='Unstim',cell_type=cell_type,patient_excluded=['P01'])
        path_stim_list=create_list_of_paths2(directory='PerioStudy_Complete Plates 1-13_011717 (Clone) - Gated Populations',stimulation=stim,cell_type=cell_type,patient_excluded=['P01'])
        res=concatenate_2conditions_multiple_data(path_stim_list,stim,path_unstim_list,'Unstim',"perio_data_sherlock/perio_data_sherlock"+stim+"_"+cell_type+"_train.h5ad")
        c+=1
        print('done ',c,'/',tot)


/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/var/folders/1w/qdnlgvv57jsctw3wf6pt0wlr0000gn/T/ipykernel_36986/1453612191.py:9: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest

done  1 / 45


/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/var/folders/1w/qdnlgvv57jsctw3wf6pt0wlr0000gn/T/ipykernel_36986/1453612191.py:9: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest

done  2 / 45


/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/var/folders/1w/qdnlgvv57jsctw3wf6pt0wlr0000gn/T/ipykernel_36986/1453612191.py:9: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest

...

done  43 / 45


/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/var/folders/1w/qdnlgvv57jsctw3wf6pt0wlr0000gn/T/ipykernel_36986/1453612191.py:9: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  comb_stim_anndata = comb_stim_anndata.concatenate(stim_anndata)
/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/var/folders/1w/qdnlgvv57jsctw3wf6pt0wlr0000gn/T/ipykernel_36986/1453612191.py:9: FutureWarning: Use anndata.concat instead of AnnData.concatenate, Ann

done  44 / 45


/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/var/folders/1w/qdnlgvv57jsctw3wf6pt0wlr0000gn/T/ipykernel_36986/1453612191.py:9: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest

done  45 / 45


In [9]:
path_unstim_pred=create_list_of_paths_spec_patients(directory='datasets/PTB drug screen complete - Gated Populations',stimulation='Unstim',cell_type='cMC',patient='HV01')
path_stim_pred=create_list_of_paths_spec_patients(directory='datasets/PTB drug screen complete - Gated Populations',stimulation='LPS',cell_type='cMC',patient='HV01')
res=concatenate_2conditions_multiple_data(path_stim_pred,'LPS',path_unstim_pred,'Unstim',"datasets/PTB_training/LPS_cMC_HV01.h5ad")

/Users/MacBook/miniconda3/envs/cellot_env/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
... storing 'drug' as categorical
... storing 'cell_type' as categorical


### Pipeline for evaluation

In [116]:
eval_4i=pd.read_csv("results/LPS_cMC/model-cellot/evals_iid_data_space/evals.csv")

In [117]:
print('eval metric available:',eval_4i['metric'].unique())

eval metric available: ['l2-means' 'l2-stds' 'r2-means' 'r2-stds' 'r2-pairwise_feat_corrs'
 'l2-pairwise_feat_corrs' 'mmd' 'enrichment-k50' 'enrichment-k100']


In [118]:
eval_4i.ncells.unique()

array([ 100,  250,  500, 1000, 1500])

In [2]:
EVALUATION_MARKERS = [
    '151Eu_pp38', '155Gd_pS6', '149Sm_pCREB', 
    '159Tb_pMAPKAPK2', '166Er_pNFkB', '167Er_pERK12'
]

In [ ]:
PTB_EVALUATION_MARKERS = ['151Eu_pP38', '155Gd_pS6', '149Sm_pCREB', '159Tb_pMK2', '166Er_pNFkB', '167Er_pERK']

train on CMC, pp38, pS6, LPS

In [148]:
original_path = "datasets/PTB_training/combined_LPS_cMC.h5ad"
target = ad.read(original_path)
target1 = target[:, '151Eu_pP38'].copy()

In [154]:
target1[target1.obs['condition'] == 'control'].X.flatten()

ArrayView([2.0405424, 1.9779768, 0.6029839, ..., 0.8639047, 0.902362 ,
           1.681758 ], dtype=float32)

In [ ]:
for marker in PTB_EVALUATION_MARKERS:
    plot_result('results/PTB/LPS_cMC/model-cellot/pred.csv','datasets/PTB_training/LPS_cMC_HV01.h5ad',marker,f"results/PTB/LPS_cMC/model-cellot/unseen_plot/LPS_cMC_HV01_{marker}.png")

In [33]:
!pip show anndata


Name: anndata
Version: 0.10.9
Summary: Annotated data.
Home-page: https://github.com/scverse/anndata
Author: Philipp Angerer, Alex Wolf, Isaac Virshup, Sergei Rybakov
Author-email: 
License: 
Location: /Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages
Requires: array-api-compat, exceptiongroup, h5py, natsort, numpy, packaging, pandas, scipy
Required-by: 


In [32]:
perio=  ad.read("perio_data_sherlock/perio_data_sherlockIFNa_B-Cells_(CD19+CD3-)_train.h5ad")
perio.keys()

/Users/olivierf/miniconda3/envs/flowcytometry/lib/python3.9/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


AttributeError: 'AnnData' object has no attribute 'keys'